In [2]:
import ur_analytic_ik
from ur_analytic_ik import ur5e
import numpy as np
np.set_printoptions(precision=4, suppress=True) # Prettier printing

In [3]:
eef_pose0 = ur5e.forward_kinematics(0, 0, 0, 0, 0, 0)
eef_pose0


array([[ 1.    ,  0.    ,  0.    , -0.8172],
       [ 0.    ,  0.    , -1.    , -0.2329],
       [ 0.    ,  1.    ,  0.    ,  0.0628],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [4]:
theta45 = np.deg2rad(45)
eef_pose45 = ur5e.forward_kinematics(theta45, theta45, theta45, theta45, theta45, theta45)
eef_pose45

array([[-0.25  , -0.4571,  0.8536,  0.0166],
       [-0.9571,  0.25  , -0.1464, -0.2715],
       [-0.1464, -0.8536, -0.5   , -0.5095],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [5]:
%%timeit
ur5e.forward_kinematics(theta45, theta45, theta45, theta45, theta45, theta45)

2.28 μs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
eef_pose = np.array(eef_pose0)
print(type(eef_pose), type(eef_pose0))
print(eef_pose.shape, eef_pose0.shape)
print(eef_pose.dtype, eef_pose0.dtype)

ur_analytic_ik.inspect(eef_pose) # works
# ur_analytic_ik.inspect(eef_pose0) # error

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(4, 4) (4, 4)
float64 float64
Tensor data pointer : 0x5627d6d8ab10
Tensor dimension : 2
Tensor dimension [0] : 4
Tensor stride    [0] : 4
Tensor dimension [1] : 4
Tensor stride    [1] : 1
Device ID = 0 (cpu=1, cuda=0)
Tensor dtype check: int16=0, uint32=0, float32=0, double=1


In [7]:
eef_pose = np.array(eef_pose0)
print("All joints should be:", 0)
solutions = ur5e.inverse_kinematics(eef_pose)
solutions

All joints should be: 0


[array([ 0.,  0.,  0., -0.,  0.,  0.]),
 array([-2.8182,  3.1416,  0.    , -3.1416, -2.8182, -0.    ]),
 array([-2.8182, -3.1416, -0.    ,  3.1416, -2.8182, -0.    ])]

In [8]:
def filter_solutions(solutions):
    filtered = [solution for solution in solutions if not np.isnan(solution).any()]
    return filtered

filter_solutions(solutions)

[array([ 0.,  0.,  0., -0.,  0.,  0.]),
 array([-2.8182,  3.1416,  0.    , -3.1416, -2.8182, -0.    ]),
 array([-2.8182, -3.1416, -0.    ,  3.1416, -2.8182, -0.    ])]

In [9]:
unique_solutions = np.unique(filter_solutions(solutions), axis=0)
unique_solutions

array([[-2.8182, -3.1416, -0.    ,  3.1416, -2.8182, -0.    ],
       [-2.8182,  3.1416,  0.    , -3.1416, -2.8182, -0.    ],
       [ 0.    ,  0.    ,  0.    , -0.    ,  0.    ,  0.    ]])

In [10]:
%%timeit
np.unique(filter_solutions(solutions), axis=0)

64.1 μs ± 4.07 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%%timeit
filter_solutions(solutions)

5.31 μs ± 1.52 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
def filter_solutions2(solutions):
    filtered = [solution for solution in solutions if not np.isnan(np.sum(solution))]
    return filtered

filter_solutions2(solutions)

[array([ 0.,  0.,  0., -0.,  0.,  0.]),
 array([-2.8182,  3.1416,  0.    , -3.1416, -2.8182, -0.    ]),
 array([-2.8182, -3.1416, -0.    ,  3.1416, -2.8182, -0.    ])]

In [13]:
%%timeit
filter_solutions2(solutions)

9.5 μs ± 230 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
solution8 = solutions[-1]
solution8

array([-2.8182, -3.1416, -0.    ,  3.1416, -2.8182, -0.    ])

In [15]:
solution8.shape

(6,)

In [16]:
np.rad2deg(solution8)

array([-161.4712, -180.    ,   -0.    ,  180.    , -161.4712,   -0.    ])

In [17]:
ur5e.forward_kinematics(*solution8.T)

array([[ 1.    ,  0.    , -0.    , -0.8172],
       [ 0.    , -0.    , -1.    , -0.2329],
       [-0.    ,  1.    , -0.    ,  0.0628],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [18]:
eef_pose = np.array(eef_pose45)
print("All joints should be:", np.deg2rad(45))
ur5e.inverse_kinematics(eef_pose)

All joints should be: 0.7853981633974483


[array([0.7854, 0.7854, 0.7854, 0.7854, 0.7854, 0.7854]),
 array([ 0.7854,  1.5375, -0.7854,  1.604 ,  0.7854,  0.7854]),
 array([ 0.7854,  0.5377,  1.5846, -2.9077, -0.7854, -2.3562]),
 array([ 0.7854,  2.041 , -1.5846, -1.2418, -0.7854, -2.3562]),
 array([-1.3058,  1.2622,  1.0985, -0.1594,  2.4741, -0.6521]),
 array([-1.3058,  2.3116, -1.0985,  0.9882,  2.4741, -0.6521]),
 array([-1.3058,  1.441 ,  1.3301,  2.5718, -2.4741,  2.4895]),
 array([-1.3058,  2.7081, -1.3301, -2.3184, -2.4741,  2.4895])]

In [19]:
%%timeit
ur5e.inverse_kinematics(eef_pose)

20.1 μs ± 584 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
ur5e.inverse_kinematics(eef_pose)

[array([0.7854, 0.7854, 0.7854, 0.7854, 0.7854, 0.7854]),
 array([ 0.7854,  1.5375, -0.7854,  1.604 ,  0.7854,  0.7854]),
 array([ 0.7854,  0.5377,  1.5846, -2.9077, -0.7854, -2.3562]),
 array([ 0.7854,  2.041 , -1.5846, -1.2418, -0.7854, -2.3562]),
 array([-1.3058,  1.2622,  1.0985, -0.1594,  2.4741, -0.6521]),
 array([-1.3058,  2.3116, -1.0985,  0.9882,  2.4741, -0.6521]),
 array([-1.3058,  1.441 ,  1.3301,  2.5718, -2.4741,  2.4895]),
 array([-1.3058,  2.7081, -1.3301, -2.3184, -2.4741,  2.4895])]

In [21]:
ur5e.inverse_kinematics.__doc__

"inverse_kinematics(arg: numpy.ndarray[dtype=float64, shape=(4, 4), order='C'], /) -> list[numpy.ndarray[dtype=float64, shape=(6), order='C']]"